# Limpieza de datos y cálculos previos

#### ¿Qué hacemos en este cuaderno?
Ordenar la información en bruto que INEI envió por correo. Esta corresponde principalmente a límites de las *ciudades principales*. 
Lo que haremos es calcular la población de cada parte de los polígonos urbanos de las *ciudades principales*.

#### ¿Qué procedimiento seguimos?
Básicamente, intersecciones entre distritos y polígonos urbanos.

#### ¿Qué datos obtenemos?
1. Un gdf que tiene solo las manchas urbanas repartidas por distritos (*ciudades_interseccion.gpkg*). Aquí si nos interesa su geometría.
2. Un gdf que tiene partidas las manchas urbanas por distritos y la población que la conforma (*ciudades_pob.geojson*). No nos interesa su geometría, sino sus datos.

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

El primer paso será realizar una intersección entre los polígonos urbanos y los distritos para saber cómo están distribuidos.

In [2]:
pu=gpd.read_file('C:/Users/rojas/OneDrive/OneDrive - EduCorpPERU/Datos/vectores/base/Manzanas Perú (INEI)/pu-principalesciudades-og_WGS84.gpkg') 
dist=gpd.read_file('C:/Users/rojas/OneDrive/OneDrive - EduCorpPERU/Datos/vectores/base/lim-dist_WGS84.gpkg')

pu=pu.to_crs('EPSG:3857') #Pseudo-Mercator
dist=dist.to_crs('EPSG:3857')

In [3]:
inter=pu.overlay(dist, how='intersection')
inter=inter[['CIUDAD','NOMBDIST','CODUBIGEO','IDPROV','NOMBPROV','geometry']] #Nos quedamos solo con las columnas que nos interesan
inter['ID'] = inter.index + 1 

In [4]:
inter.head()

,CIUDAD,NOMBDIST,CODUBIGEO,IDPROV,NOMBPROV,geometry,ID
0,ABANCAY,TAMBURCO,30109,0301,ABANCAY,"MULTIPOLYGON (((-8112020.618 -1529196.326, -81...",1
1,ABANCAY,ABANCAY,30101,0301,ABANCAY,"MULTIPOLYGON (((-8112178.097 -1531791.118, -81...",2
2,ANDAHUAYLAS,TALAVERA,30216,0302,ANDAHUAYLAS,"POLYGON ((-8176551.463 -1532941.437, -8176530....",3
3,ANDAHUAYLAS,SAN JERONIMO,30213,0302,ANDAHUAYLAS,"POLYGON ((-8167720.999 -1534466.258, -8167729....",4
4,ANDAHUAYLAS,ANDAHUAYLAS,30201,0302,ANDAHUAYLAS,"MULTIPOLYGON (((-8172249.289 -1534344.842, -81...",5


In [5]:
inter.to_file("data/ciudades_interseccion.gpkg", driver='GPKG')

Ahora vamos a obtener el total de población por manzana.

In [6]:
gdf=gpd.read_file('C:/Users/rojas/OneDrive/OneDrive - EduCorpPERU/Datos/vectores/base/Manzanas Perú (INEI)/mz-principalesciudades-edit_WGS84.gpkg')
pob=pd.read_excel('C:/Users/rojas/OneDrive/OneDrive - EduCorpPERU/Datos/tablas/Personas x manzana - INEI.xlsx')

gdf=gdf.to_crs('EPSG:3857')

Cambiamos los nombres de las columnas para que coincidan

In [7]:
gdf = gdf.rename(columns={'Mz_Edit':'Mz'})
pob = pob.rename(columns={'Cod':'Mz'})

Unimos el gdf con los datos de población por manzana y lo convertimos en centroides para que pese menos el archivo.

In [8]:
#1.1. Hacemos el merge
gdf = gdf.merge(pob, on='Mz')

#1.2. Convertimos el gdf de manzanas en centroides
centros=gpd.GeoDataFrame(gdf.centroid)
centros.rename(columns={0:'geometry'}, 
                inplace=True)                      #para setear el CRS es necesario que la única columna que tiene
                                                   #se llame 'geometry'
    
centros=centros.set_crs('EPSG:3857')              #para que tenga el mismo CRS que el gdf al que se unirá

#1.3. Ahora los centros deben 'absorber' los datos de su gdf original
mz = gpd.sjoin(centros, gdf, how="left")
mz.head()

,geometry,index_right,Mz_Original,Mz,CIUDAD,UBIGEO,CODPROV,CODDPTO,NOMBDIST,NOMBPROV,NOMBDPTO,Población
0,POINT (-8669524.377 -693808.991),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,POINT (-8669371.073 -694019.822),1.0,01010100100001K,010101000100100001K,CHACHAPOYAS,010101,01,01,CHACHAPOYAS,CHACHAPOYAS,AMAZONAS,70.0
2,POINT (-8669378.964 -694067.903),2.0,01010100100001L,010101000100100001L,CHACHAPOYAS,010101,01,01,CHACHAPOYAS,CHACHAPOYAS,AMAZONAS,62.0
3,POINT (-8669350.380 -694165.716),3.0,01010100100002D,010101000100100002D,CHACHAPOYAS,010101,01,01,CHACHAPOYAS,CHACHAPOYAS,AMAZONAS,42.0
4,POINT (-8669293.824 -694264.816),4.0,01010100100003B,010101000100100003B,CHACHAPOYAS,010101,01,01,CHACHAPOYAS,CHACHAPOYAS,AMAZONAS,70.0


Nos quedamos con las columnas que nos interesan.

In [9]:
mz=mz[['geometry','Mz','UBIGEO','Población']]

Hacemos el spatial join de los centroides de las manzanas (que tienen info de población) con las manchas urbanas distribuidas por 'pedacitos' (la intersección del inicio).

In [10]:
intento1=gpd.sjoin(mz,inter,how="left", op="within")   #por defecto, 'sjoin' utiliza la opción 'intersect'
                                                         #sin embargo, al usar 'within' los resultados pueden ser hasta
                                                         #10 veces más rápido
intento1.head()

C:\Users\rojas\miniconda3\envs\MiEntorno\lib\site-packages\IPython\core\interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


,geometry,Mz,UBIGEO,Población,index_right,CIUDAD,NOMBDIST,CODUBIGEO,IDPROV,NOMBPROV,ID
0,POINT (-8669524.377 -693808.991),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,POINT (-8669371.073 -694019.822),010101000100100001K,010101,70.0,47.0,CHACHAPOYAS,CHACHAPOYAS,10101.0,0101,CHACHAPOYAS,48.0
2,POINT (-8669378.964 -694067.903),010101000100100001L,010101,62.0,47.0,CHACHAPOYAS,CHACHAPOYAS,10101.0,0101,CHACHAPOYAS,48.0
3,POINT (-8669350.380 -694165.716),010101000100100002D,010101,42.0,47.0,CHACHAPOYAS,CHACHAPOYAS,10101.0,0101,CHACHAPOYAS,48.0
4,POINT (-8669293.824 -694264.816),010101000100100003B,010101,70.0,47.0,CHACHAPOYAS,CHACHAPOYAS,10101.0,0101,CHACHAPOYAS,48.0


In [11]:
len(intento1)

116375

In [12]:
#3.Filtramos solo aquellas columnas que estén dentro de las manchas urbanas de las ciudades principales
filtro=intento1['ID']>=0
intento2 = intento1[filtro]

intento2.to_file("data/ciudades_pob.geojson", driver='GeoJSON') 
#Por algún motivo, al guardarlo como geopackage, se creaban dos capas, y una de ellas omitía a las ciudades
#de Ayacucho, Chancay, Huanta y Huaral.

intento2.head()

,geometry,Mz,UBIGEO,Población,index_right,CIUDAD,NOMBDIST,CODUBIGEO,IDPROV,NOMBPROV,ID
1,POINT (-8669371.073 -694019.822),010101000100100001K,010101,70.0,47.0,CHACHAPOYAS,CHACHAPOYAS,10101.0,0101,CHACHAPOYAS,48.0
2,POINT (-8669378.964 -694067.903),010101000100100001L,010101,62.0,47.0,CHACHAPOYAS,CHACHAPOYAS,10101.0,0101,CHACHAPOYAS,48.0
3,POINT (-8669350.380 -694165.716),010101000100100002D,010101,42.0,47.0,CHACHAPOYAS,CHACHAPOYAS,10101.0,0101,CHACHAPOYAS,48.0
4,POINT (-8669293.824 -694264.816),010101000100100003B,010101,70.0,47.0,CHACHAPOYAS,CHACHAPOYAS,10101.0,0101,CHACHAPOYAS,48.0
5,POINT (-8669341.131 -694221.467),010101000100100005,010101,45.0,47.0,CHACHAPOYAS,CHACHAPOYAS,10101.0,0101,CHACHAPOYAS,48.0


In [13]:
len(intento2)

116323